In [1]:
import psycopg2
import pandas as pd

In [2]:
conn = psycopg2.connect(host='localhost',
                        dbname='sql_practice',
                        user='postgres',
                        password='Aujaswi@31',
                        port=5432)

In [3]:
conn.autocommit = True

In [4]:
cur = conn.cursor()

# Question 
### A table named “famous” has two columns called user id and follower id. It represents each user ID has a particular follower ID. These follower IDs are also users of hashtag#Facebook / hashtag#Meta. Then, find the famous percentage of each user. Famous Percentage = number of followers a user has / total number of users on the platform.

## Creating the table

In [5]:
cur.execute("""CREATE TABLE IF NOT EXISTS famous (user_id INT, follower_id INT);
""")

## Populating the table

In [6]:
cur.execute("""INSERT INTO famous VALUES
(1, 2), (1, 3), (2, 4), (5, 1), (5, 3), 
(11, 7), (12, 8), (13, 5), (13, 10), 
(14, 12), (14, 3), (15, 14), (15, 13);""")

### Solution

In [7]:
cur.execute("""WITH unique_users AS (
            SELECT DISTINCT(user_id) AS users FROM famous
            UNION
            SELECT DISTINCT(follower_id) AS users FROM famous),
            followers AS (SELECT user_id, COUNT(DISTINCT follower_id) follower_count FROM famous GROUP BY user_id)
            SELECT f.user_id user_id, ROUND(f.follower_count * 100.0 / (SELECT COUNT(*) FROM unique_users),2) famous_percentage FROM followers f """)

In [8]:
df = pd.DataFrame(cur.fetchall(),columns=['user_id','famous_percentage'])

In [9]:
df

,user_id,famous_percentage
0,1,15.38
1,2,7.69
2,5,15.38
3,11,7.69
4,12,7.69
5,13,15.38
6,14,15.38
7,15,15.38


In [10]:
cur.execute("""DROP TABLE famous;""")

In [11]:
cur.close()
conn.close()